In [ ]:
# mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/COS429_Final_Project/

/content/drive/.shortcut-targets-by-id/1ovw9re0A-fNM5W9dPWViPUskeUdZQ7j6/COS429_Final_Project


In [ ]:
import os

In [ ]:
# HOW TO USE: If you have a folder with classes and image frames in those classes and want to create a train/val split

# new data folder name to contain train/val split folders
new_folder_name = 'BOSTON_DATA_28_HANDS_TRAINVAL_V2' # folder name to create train val and put the folders in there
dest = new_folder_name
source = "BOSTON_DATA_28_TRAINVAL" # where the frames are atm (not in train/val split)
train_test_split = 0.3
anno_folder = "BOSTON_DATA_28_ANNOHANDS_TRAINVAL_V2"

In [ ]:
label_names = os.listdir(source+'/train')

class_name = {}
for i, label in enumerate(label_names):
  class_name[label] = i

"label_names = os.listdir(source+'/train')\n\nclass_name = {}\nfor i, label in enumerate(label_names):\n  class_name[label] = i"

In [ ]:

os.system('mkdir '+ new_folder_name)
os.system('mkdir '+ new_folder_name+'/train')
os.system('mkdir '+ new_folder_name+'/val')

os.system('mkdir '+ anno_folder)
os.system('mkdir '+ anno_folder+'/train')
os.system('mkdir '+ anno_folder+'/val')

"\nos.system('mkdir '+ new_folder_name)\nos.system('mkdir '+ new_folder_name+'/train')\nos.system('mkdir '+ new_folder_name+'/val')\n\nos.system('mkdir '+ anno_folder)\nos.system('mkdir '+ anno_folder+'/train')\nos.system('mkdir '+ anno_folder+'/val')"

In [ ]:
# Minimum dependency for MediaPipe Solutions Python API is opencv-python
!pip install opencv-python~=3.4.11
!pip install mediapipe

In [ ]:
import math
from typing import List, Tuple, Union

import cv2
import dataclasses
import numpy as np

from mediapipe.framework.formats import landmark_pb2

RGB_CHANNELS = 3
RED_COLOR = (0, 0, 255)


@dataclasses.dataclass
class DrawingSpec:
  # Color for drawing the annotation. Default to the green color.
  color: Tuple[int, int, int] = (0, 255, 0)
  # Thickness for drawing the annotation. Default to 2 pixels.
  thickness: int = 2
  # Circle radius. Default to 2 pixels.
  circle_radius: int = 2


def _normalized_to_pixel_coordinates(
    normalized_x: float, normalized_y: float, image_width: int,
    image_height: int) -> Union[None, Tuple[int, int]]:
  """Converts normalized value pair to pixel coordinates."""

  # Checks if the float value is between 0 and 1.
  def is_valid_normalized_value(value: float) -> bool:
    return (value > 0 or math.isclose(0, value)) and (value < 1 or
                                                      math.isclose(1, value))

  if not (is_valid_normalized_value(normalized_x) and
          is_valid_normalized_value(normalized_y)):
    # TODO: Draw coordinates even if it's outside of the image bounds.
    return None
  x_px = min(math.floor(normalized_x * image_width), image_width - 1)
  y_px = min(math.floor(normalized_y * image_height), image_height - 1)
  return x_px, y_px

In [ ]:
# For static images:
import cv2
import mediapipe as mp
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.05)

from google.protobuf.json_format import MessageToDict

def vconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    w_min = min(im.shape[1] for im in im_list)
    im_list_resize = [cv2.resize(im, (w_min, int(im.shape[0] * w_min / im.shape[1])), interpolation=interpolation)
                      for im in im_list]
    return cv2.vconcat(im_list_resize)

def get_crop_info(left, right, top, bottom, padding, image_rows, image_cols):
  if left-padding < 0:
    left = 0
  else:
    left = left-padding

  if top-padding < 0:
    top = 0
  else:
    top = top-padding

  if right+padding > image_rows:
    right = image_rows
  else:
    right = right+padding

  if bottom+padding > image_cols:
    bottom = image_cols
  else:
    bottom = bottom+padding
  
  y = min(int(top), int(bottom))
  h = int(abs(top-bottom))

  x = min(int(left), int(right))
  w = int(abs(left-right))
  return x, y, w, h

def get_crop_img(image, x, y, w, h):
  crop_img = image[int(y):int(y+h), int(x):int(x+w)]
  return crop_img

def extract_hand(file_name, image_write_path, anno_image_write_path, padding=50):
  # Read an image, flip it around y-axis for correct handedness output (see
  # above).
  image = cv2.flip(cv2.imread(file_name), 1)
  # Convert the BGR image to RGB before processing.
  # results = hands.process(image)
  hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.05)
  results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

  x_0= []
  y_0 = []

  x_1= []
  y_1 = []

  idx_to_coordinates = {}
  image_rows, image_cols, _ = image.shape

  # Print handedness and draw hand landmarks on the image.
  #print('handedness:', results.multi_handedness)
  if not results.multi_hand_landmarks:
    # write back the image image
    cv2.imwrite(image_write_path, image)
    #cv2.imwrite(anno_image_write_path, image)
    return

  #MessageToDict(hand_handedness)
  #annotated_image_0 = image.copy() #right 
  #annotated_image_1 = image.copy() #left 
  hands = len(results.multi_hand_landmarks)
  blank_image = np.zeros((image_rows,image_rows,3), np.uint8)

  for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
    handedness = MessageToDict(results.multi_handedness[i])['classification'][0]['label']
    for idx, landmark in enumerate(hand_landmarks.landmark):
      landmark_px = _normalized_to_pixel_coordinates(landmark.x, landmark.y,
                                                   image_cols, image_rows)
      if landmark_px !=None:
        if handedness=='Right':
          x_0.append(landmark_px[0])
        else:
          x_1.append(landmark_px[0])

        if handedness=='Right':
          y_0.append(landmark_px[1])
        else:
          y_1.append(landmark_px[1])

    '''if handedness=='Right':
       mp_drawing.draw_landmarks(annotated_image_0, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    if handedness=='Left':
       mp_drawing.draw_landmarks(annotated_image_1, hand_landmarks, mp_hands.HAND_CONNECTIONS)'''
    
  if x_0 == []: # no right hand
    crop_img0 = blank_image
    #crop_anno_img0 = blank_image
  else:
    left0, right0, top0, bottom0 = min(x_0), max(x_0), min(y_0), max(y_0)
    x0, y0, w0, h0= get_crop_info(left0, right0, top0, bottom0, padding, image_rows, image_cols)
    w0 = max(w0, h0)
    crop_img0 = get_crop_img(image, x0, y0, w0, w0)
    #crop_anno_img0 = get_crop_img(annotated_image_0, x0, y0, w0, w0)

  if x_1 == []: # no right hand
    crop_img1 = blank_image
    #crop_anno_img1 = blank_image
  else:
    left1, right1, top1, bottom1 = min(x_1), max(x_1), min(y_1), max(y_1)
    x1, y1, w1, h1= get_crop_info(left1, right1, top1, bottom1, padding, image_rows, image_cols)
    w1 = max(w1, h1)
    crop_img1 = get_crop_img(image, x1, y1, w1, w1)
    #crop_anno_img1 = get_crop_img(annotated_image_1, x1, y1, w1, w1)


  if x_1 == [] and x_0==[]:
    hand = image
    #hand_anno = image
  else:
    hand = vconcat_resize_min([crop_img0, crop_img1])
    #hand_anno = vconcat_resize_min([crop_anno_img0, crop_anno_img1])

  cv2.imwrite(image_write_path, cv2.flip(hand, 1))
  #cv2.imwrite(anno_image_write_path, cv2.flip(hand_anno, 1))
  return


In [ ]:
#extract_hand('/content/13855-0.jpg', '/content/img_hand_new.png', '/content/anno_img_hand_new.png', padding=50)

In [ ]:
train_path= '/content/drive/MyDrive/COS429_Final_Project/'+ source+ '/train/'

for c_name in class_name.keys():
  #os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ anno_folder + '/train/' + c_name)
  os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name + '/train/' + c_name)
  print(c_name)
  for folder_name in os.listdir(train_path+c_name):
    #os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ anno_folder + '/train/' + c_name+ '/'+folder_name)
    os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name + '/train/' + c_name+ '/'+folder_name)

    folder_path = '/content/drive/MyDrive/COS429_Final_Project/'+ source+ '/train/' + c_name+ '/'+folder_name
    print(folder_name)
    for image_name in os.listdir(folder_path):
      image_write_path = '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name+ '/train/' + c_name+ '/'+folder_name+'/'+image_name
      anno_image_write_path = '/content/drive/MyDrive/COS429_Final_Project/'+ anno_folder+ '/train/' + c_name+ '/'+folder_name+'/'+image_name
      image_path = '/content/drive/MyDrive/COS429_Final_Project/'+ source + '/train/' + c_name+ '/'+folder_name+'/'+image_name
      extract_hand(image_path, image_write_path, anno_image_write_path)

"train_path= '/content/drive/MyDrive/COS429_Final_Project/'+ source+ '/train/'\n\nfor c_name in class_name.keys():\n  #os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ anno_folder + '/train/' + c_name)\n  os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name + '/train/' + c_name)\n  print(c_name)\n  for folder_name in os.listdir(train_path+c_name):\n    #os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ anno_folder + '/train/' + c_name+ '/'+folder_name)\n    os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name + '/train/' + c_name+ '/'+folder_name)\n\n    folder_path = '/content/drive/MyDrive/COS429_Final_Project/'+ source+ '/train/' + c_name+ '/'+folder_name\n    print(folder_name)\n    for image_name in os.listdir(folder_path):\n      image_write_path = '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name+ '/train/' + c_name+ '/'+folder_name+'/'+image_name\n      anno_image_wr

In [ ]:
val_path= '/content/drive/MyDrive/COS429_Final_Project/'+ source+ '/val/'

for c_name in class_name.keys():
  os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ anno_folder + '/val/' + c_name)
  os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name + '/val/' + c_name)
  print(c_name)
  for folder_name in os.listdir(val_path+c_name):
    os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ anno_folder + '/val/' + c_name+ '/'+folder_name)
    os.system('mkdir '+ '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name + '/val/' + c_name+ '/'+folder_name)

    folder_path = '/content/drive/MyDrive/COS429_Final_Project/'+ source+ '/val/' + c_name+ '/'+folder_name
    print(folder_name)
    for image_name in os.listdir(folder_path):
      image_write_path = '/content/drive/MyDrive/COS429_Final_Project/'+ new_folder_name+ '/val/' + c_name+ '/'+folder_name+'/'+image_name
      anno_image_write_path = '/content/drive/MyDrive/COS429_Final_Project/'+ anno_folder+ '/val/' + c_name+ '/'+folder_name+'/'+image_name
      image_path = '/content/drive/MyDrive/COS429_Final_Project/'+ source + '/val/' + c_name+ '/'+folder_name+'/'+image_name
      extract_hand(image_path, image_write_path, anno_image_write_path)

CRASH
302933
305575
305211
FAVORITE
263488
51850
51875
299658
301352
307281
FRIEND
13577
13862
301376
300678
13794
300650
301214
300916
299707
263791
302790
299168
299505
300939
300976
300792
14529
203474
13545
13928
48267
203334
203422
203430
203573
203641
